# Clase 25 - Python Driver
### Sofía Maldonado García

In [17]:
# Imports
import pandas as pd

from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

In [18]:
# Conexión
uri = 'bolt://localhost:7687'
user = 'neo4j' #Es el default
password = 'chofas1973' #Depende de cada DBMS

with GraphDatabase.driver(uri, auth=(user,password)) as driver:
    driver.verify_connectivity()

# Si el código ejecuta sin errores, significa que la conexión fue exitosa

In [27]:
# Consulta a la base de datos

query = """
    MATCH (p:Product)
    RETURN p.productName AS Nombre, p.unitPrice AS Precio
    ORDER BY Precio DESC
    LIMIT 10;
"""

rows = None

try:
    with driver.session() as session:
        result = session.run(query)
        rows = result.data()

        print('Nombres de Productos: \n')
        for i in rows:
            print(f'Producto: {i['Nombre']} \nPrecio: ${i['Precio']}')
except Exception as e:
    print(f'Error: {e}')

Nombres de Productos: 

Producto: Côte de Blaye 
Precio: $263.5
Producto: Thüringer Rostbratwurst 
Precio: $123.79
Producto: Mishi Kobe Niku 
Precio: $97.0
Producto: Sir Rodney's Marmalade 
Precio: $81.0
Producto: Carnarvon Tigers 
Precio: $62.5
Producto: Raclette Courdavault 
Precio: $55.0
Producto: Manjimup Dried Apples 
Precio: $53.0
Producto: Tarte au sucre 
Precio: $49.3
Producto: Ipoh Coffee 
Precio: $46.0
Producto: Rössle Sauerkraut 
Precio: $45.6


C:\Users\pixta\AppData\Local\Temp\ipykernel_20952\4027871894.py:13: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


#### Conexión con GraphDataScience

In [30]:
# uri = 'bolt://localhost:7687'
# user = 'neo4j' #Es el default
# password = 'chofas1973' #Depende de cada DBMS

gds = GraphDataScience(uri,auth=(user,password))
print(gds.server_version()) # Me va a retornar 2.13.2, la version de GDS que hemos estado usando

2.13.2


In [35]:
# Coolness 

query = """
    MATCH (p:Product)
    RETURN p.productName AS Nombre, p.unitPrice AS Precio
    ORDER BY Precio DESC;
"""

final = gds.run_cypher(query)
final

,Nombre,Precio
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
...,...,...
72,Tourtière,7.45
73,Filo Mix,7.00
74,Konbu,6.00
75,Guaraná Fantástica,4.50


In [37]:
# Crear subgrafo no dirigido

gds.run_cypher("""
    call gds.graph.project(
    'myGraph5',
    'Product',
    {BUY_TOGETHER: {orientation:'UNDIRECTED'}}
)
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Product': {'label': 'Product', 'properties':...","{'BUY_TOGETHER': {'aggregation': 'DEFAULT', 'o...",myGraph5,77,6140,38


In [38]:
# Correr un algoritmo

gds.run_cypher("""
    call gds.louvain.write('myGraph5', {writeProperty: 'louvain_3'})
    YIELD communityCount, modularity, modularities;
""")

,communityCount,modularity,modularities
0,4,0.100507,[0.1005069549809547]


In [43]:
gds.run_cypher("""
    MATCH (p:Product)-[r:BUY_TOGETHER]->(op:Product)
    WHERE p.productName > op.productName
    RETURN p.productName AS Producto1, op.productName AS Producto2, r.quantity AS Veces, p.louvain_3 AS LV1, op.louvain_3 AS LV2
    ORDER BY Veces DESC
    LIMIT 5; 
""")

,Producto1,Producto2,Veces,LV1,LV2
0,Sirop d'érable,Sir Rodney's Scones,8,49,49
1,Pavlova,Gorgonzola Telino,7,65,49
2,Tarte au sucre,Pavlova,6,36,65
3,Tourtière,Nord-Ost Matjeshering,6,36,36
4,Mozzarella di Giovanni,Gorgonzola Telino,6,36,49
